# Model devepment

In this document we develop and compare different models

TODO: write a nicer intro

### Import libraries

In [1]:
#from preprocessCHECK import preprocessor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score
from scipy.stats import uniform, randint

### Data collection

In [2]:
# Data collection
total_df = pd.read_csv('./Data/Base.csv')

# Define features (X) and target (y)
X = total_df.drop(columns=['fraud_bool'])
y = total_df['fraud_bool']

# Split the data into training and test sets using stratified sampling
train_df, test_df = train_test_split(total_df, test_size=0.2, stratify=total_df['fraud_bool'], random_state=42)

In [3]:
# Create numerical dataframe
num_df = train_df.select_dtypes(include=['float64']).drop(columns=['income', 'proposed_credit_limit'])

# Create categorical dataframe
cat_df = train_df.select_dtypes(include=['int64', 'object']).copy()
cat_df[['income', 'proposed_credit_limit']] = train_df[['income', 'proposed_credit_limit']]
cat_df['income'] = cat_df['income'].round(1)
cat_df['proposed_credit_limit'] = cat_df['proposed_credit_limit'].round(0).astype('int64')


highcard_df = cat_df[[col for col in cat_df.columns if cat_df[col].nunique() > 12]]
lowcard_df = cat_df[[col for col in cat_df.columns if (cat_df[col].nunique() <= 12) & (cat_df[col].nunique() > 2)]]
bool_df = cat_df[[col for col in cat_df.columns if cat_df[col].nunique() <= 2]]


In [ ]:
from preprocessingwip import numerical_pipeline
from preprocessingwip import low_card_pipeline
from preprocessingwip import high_card_pipeline
from preprocessingwip import boolean_pipeline
from preprocessingwip import resampling_pipeline

transformed_num = numerical_pipeline.fit_transform(num_df)
transformed_highcard = high_card_pipeline.fit_transform(highcard_df)
transformed_lowcard = low_card_pipeline.fit_transform(lowcard_df)
transformed_boolean = boolean_pipeline.fit_transform(bool_df)

# NEARMISS RESAMPLING
t_num_X_resampled, t_num_y_resampled = resampling_pipeline(transformed_num, y)

# X_resampled, y_resampled = resampling_pipeline(X_preprocessed, y)

# feature_selection(X_resampled, y_resampled)

# variance_threshold_test()



In [ ]:
from preprocessCHECK import numerical_preprocessor

if __name__ == "__main__":
    # Sample data for testing
    sample_df = pd.DataFrame({
        'name_email_similarity': np.random.rand(10),
        'days_since_request': np.random.randint(1, 1000, size=10),
        'intended_balcon_amount': np.random.randint(-1, 100, size=10),
        'velocity_6h': np.random.randint(-1, 50, size=10),
        'velocity_24h': np.random.randint(0, 100, size=10),
        'velocity_4w': np.random.rand(10),
        'session_length_in_minutes': np.random.rand(10),
        # Add other columns as needed
    })

    # Fit and transform
    numerical_preprocessor.fit(sample_df)
    X_transformed = numerical_preprocessor.transform(sample_df)
    
    print("Transformed X shape:", X_transformed.shape)
    print("Transformed X:", X_transformed)

ValueError: The output of the 'days_since_request' transformer should be 2D (numpy array, scipy sparse array, dataframe).

In [52]:
from preprocessCHECK import numerical_preprocessor

# Data collection
total_df = pd.read_csv('./Data/Base.csv')

# Define features (X) and target (y)
X = total_df.drop(columns=['fraud_bool'])

print(f"Input shape: {X.shape}")
print(f"Input type: {type(X)}")
numerical_preprocessor.fit(X)
X_transformed = numerical_preprocessor.transform(X)

print("Transformed X_train shape:", X_transformed.shape)

Input shape: (1000000, 31)
Input type: <class 'pandas.core.frame.DataFrame'>


ValueError: Input array must be 1 dimensional

### Model classes

In [26]:
# Pipeline with placeholder classifier
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression())])

# Define models and their hyperparameters
models = {
    'Logistic Regression': (LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000),
                            {'C': uniform(0.01, 10)}),
    'Random Forest': (RandomForestClassifier(class_weight='balanced', random_state=42),
                      {'n_estimators': randint(50, 150), 'max_depth': randint(3, 10)}),
    'SVC': (SVC(class_weight='balanced', probability=True, random_state=42),
            {'C': uniform(0.01, 10), 'kernel': ['linear', 'rbf']}),
    'KNN': (KNeighborsClassifier(),
            {'n_neighbors': randint(3, 10)}),
    'Gradient Boosting': (GradientBoostingClassifier(random_state=42),
                          {'n_estimators': randint(50, 150), 'learning_rate': uniform(0.01, 0.2)}),
    'XGBoost': (XGBClassifier(random_state=42),
                {'n_estimators': randint(50, 150), 'learning_rate': uniform(0.01, 0.2)}),
    'LightGBM': (LGBMClassifier(random_state=42),
                 {'n_estimators': randint(50, 150), 'learning_rate': uniform(0.01, 0.2)}),
    'Naive Bayes': (GaussianNB(), {})  # No hyperparameters for Naive Bayes
}

## Hyperparameter tuning

In [ ]:
# Stratified K-Fold Cross-Validation
stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Number of random searches per model
n_iter_per_model = 1
best_models = []

# Iterate through each model
for name, (model, params) in models.items():
    print(f"Running RandomizedSearchCV for {name}...")

    # Create pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

    # Create RandomizedSearchCV
    search = RandomizedSearchCV(
        pipeline, 
        param_distributions=params,
        n_iter=n_iter_per_model,
        cv=stratified_cv,
        n_jobs=-1,
        random_state=42,
        scoring='roc_auc'
    )

    # Fit the model
    search.fit(X_train, y_train)

    # Store the best model and its score
    best_models.append((name, search.best_estimator_, search.best_score_))

# Find the overall best model
best_name, best_model, best_score = max(best_models, key=lambda x: x[2])

# Evaluate the best model
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1] if hasattr(best_model.named_steps['classifier'], 'predict_proba') else y_pred

# Print the best model and its parameters
print(f"\nBest Model: {best_name}")
print(f"Best Cross-Validation AUC Score: {best_score:.4f}")

# Print classification report
print(classification_report(y_test, y_pred))

# Calculate and print AUC score on the test set
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"AUC Score on Test Set: {auc_score:.4f}")

Running RandomizedSearchCV for Logistic Regression...
Running RandomizedSearchCV for Random Forest...


KeyboardInterrupt: 

In [23]:
X_train_transformed = preprocessor.fit_transform(X_train)
print(X_train_transformed.dtypes)

ValueError: Input array must be 1 dimensional